In [1]:
# for Google Colab only: Mount my Google Drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
pwd

'/gdrive'

In [3]:
# for Google Colab only: Navigate to my snaug project folder
cd 'My Drive/Colab Notebooks/snaug'

/gdrive/My Drive/Colab Notebooks/snaug


In [4]:
#
# Load saved weights of models that were previously trained on cloud platforms 
# using GPU (eg. Google Colab)
# Save an instance of trained final model for productionalisation
#

import string
import textwrap
import pickle

from nlplstm_class import TFModelLSTMWord2vec 
from data_common import (load_doc, save_doc, clean_doc, prepare_char_tokens)
from data_common import (build_token_lines, prepare_text_tokens, load_word2vec)
from data_common import pathfinder_textfile, fixed_length_token_textfile

Using TensorFlow backend.


In [0]:
# load fixed-length lines of tokens
doc = load_doc(fixed_length_token_textfile)
lines = doc.split('\n')
#print('Total lines: %d' % len(lines))

In [0]:
# tokenize and separate into features and target
X, y, seq_length, vocab_size, tokenizer = prepare_text_tokens(lines)
#print(X.shape)

In [0]:
# load gensim Word2Vec word model's pretrained weights
pretrained_weights = pickle.load(open('./model/pathfinder_wordtoken_w2v_word_model_weights.pkl', 'rb'))
vocab_size, emdedding_size = pretrained_weights.shape

In [0]:
# create new object that is an LSTM model using word tokenization
# and pre-trained Word2vec model form Gensim to generate text
# this model does not use CudaDNN library
#
textgen_final_model = TFModelLSTMWord2vec(use_gpu=False)

In [9]:
# define the model parameters
textgen_final_model.define(vocab_size=vocab_size, 
                       embedding_size=emdedding_size, 
                       pretrained_weights=pretrained_weights)
print(textgen_final_model.model.summary())











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         877500    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 300)         721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300 

In [0]:
# load model weights trained on platform using GPU
textgen_final_model.load_weights("./model/pathfinder_wordtoken_w2v_model_50_epoch")

In [0]:
# save model updated with previously trained model weights
textgen_final_model.save("./model/pathfinder_wordtoken_w2v_model_50_epoch_noncuda")